In [2]:
import pandas as pd 

In [20]:
def process_gov_csv(file_name): 
    df= pd.read_csv(file_name, delimiter="\t",encoding='utf-16')
    df.drop('Gender',axis=1,inplace=True)
    df.drop('Year of Birth',axis=1,inplace=True)
    df['Start']=pd.to_datetime(df['Start'],errors='coerce',format='%d-%b-%y')
    df['End']=pd.to_datetime(df['End'],errors='coerce',format='%d-%b-%y')
    #df['Start']=df['Start'].dt.strftime('-%b-%y')
    #df['End']=df['End'].dt.strftime('-%b-%y')
    return df 

In [3]:
file_name='data.csv'

In [21]:
df=process_gov_csv(file_name)

In [11]:
df.to_csv('example.csv')

 now, onto matching :D 

In [33]:
def matching(countryname,election_dates,df): 
    sorted_elections= sorted(pd.to_datetime(election_dates, format=f'{countryname}-%b-%y'))
    election_period_dict = {}
    for i in range((len(sorted_elections))): 
        start=sorted_elections[i]
        end = sorted_elections[i+1] if i+1<len(sorted_elections) else pd.Timestamp.today().strftime('-%b-%y')
        boolean_mask = ((df['Start']>=start)&(df['Start']<end))
        df_i=df[boolean_mask]
        key=start.strftime('-%b-%y')
        election_period_dict[key]=df_i
    return election_period_dict
        

In [13]:
election_dates = [
    "Austria-Oct-94", "Austria-Dec-95", "Austria-Oct-99", "Austria-Nov-02",
    "Austria-Oct-06", "Austria-Sep-08", "Austria-Sep-13", "Austria-Oct-17", "Austria-Sep-19"
]
countryname = 'Austria'


In [18]:
sorted_elections= sorted(pd.to_datetime(election_dates, format=f'{countryname}-%b-%y'))

In [19]:
pd.Timestamp.today().strftime('-%b-%y')

'-Feb-24'

In [32]:
voting_1=sorted_elections[5]
voting_2 = sorted_elections[5+1] if 1+1<len(sorted_elections) else pd.Timestamp.today().strftime('-%b-%y')
boolean_mask = ((df['Start']>=voting_1)&(df['Start']<voting_2))
df_i=df[boolean_mask]

In [29]:
df_1=(df['Start']>=start)&(df['Start']<end)

In [34]:
test = matching(countryname,election_dates,df)

In [36]:
test.get('-Sep-08')

,Position,Start,End,Name,Party
8,Federal Chancellor,2008-12-02,2013-12-15,Werner Faymann,SPÖ - Social Democratic Party of Austria (Soz...
20,Vice Chancellor,2011-04-21,2013-12-15,Michael Spindelegger,ÖVP - Austrian People's Party (Österreichisch...
21,Vice Chancellor,2008-12-02,2011-04-21,Josef Pröll,ÖVP - Austrian People's Party (Österreichisch...
30,"Federal Minister of Agriculture, Forestry, the...",2008-12-02,2013-12-15,Nikolaus Berlakovich,ÖVP - Austrian People's Party (Österreichisch...
51,Federal Minister of European and International...,2008-12-02,2013-12-15,Michael Spindelegger,ÖVP - Austrian People's Party (Österreichisch...
62,Federal Minister of National Defence and Sport,2013-03-11,2013-12-15,Gerald Klug,SPÖ - Social Democratic Party of Austria (Soz...
63,Federal Minister of National Defence and Sport,2008-12-02,2013-03-11,Norbert Darabos,SPÖ - Social Democratic Party of Austria (Soz...
72,"Federal Minister of Transport, Innovation and ...",2008-12-02,2013-12-15,Doris Bures,SPÖ - Social Democratic Party of Austria (Soz...
100,"Federal Minister for Education, Arts and Culture",2008-12-02,2013-12-15,Claudia Schmied,SPÖ - Social Democratic Party of Austria (Soz...
114,Federal Minister of Finance,2011-04-21,2013-12-15,Maria Fekter,ÖVP - Austrian People's Party (Österreichisch...
